# Useful Cell Culture Functions

In [89]:
import numpy as np
from datetime import datetime
from datetime import timedelta
from numbers import Number

## Calculate Doubling Time

In [2]:
def calculate_doubling_time(t1, t2, od1, od2, unit="min"):
    """Returns the doubling time of a culture given two times t1 and t2 and two OD600s od1 and od2.
    Enter time as a string (in 24 hour format)"""
    assert od1 < od2
    FMT = "%H:%M"
    time_difference = datetime.strptime(t2, FMT) - datetime.strptime(t1, FMT)
    assert time_difference.seconds > 0
    if unit == "min":
        return((time_difference.seconds/60) / np.log2(od2 / od1))
    if unit == "hour":
        return((time_difference.seconds/3600) / np.log2(od2 / od1))

In [3]:
calculate_doubling_time("18:03", "19:53", 0.065, 0.188)
calculate_doubling_time("19:53", "20:46", 0.188, 0.316)
calculate_doubling_time("20:46", "21:05", 0.316, 0.372)

80.72120130233009

## Design a Growth Experiment

In [109]:
def how_should_i_innoculate(target_od, target_time, target_volume, starter_od, time_now, \
                            doubling_time=90, print_total_elapsed=False, include_lag=False):
    """
    Instructions on how to make a culture from a yeast culture - all volume units are in mL!
    If you want to see how long your yeast will be growing for, set print_total_elasped to be True
    If you want to include a lag phase, set include_lag to be 'True' for 2 hours, or a number of hours
    For a custom value.
    """
    FMT = "%H:%M"
    time_diff = datetime.strptime(target_time, FMT) - datetime.strptime(time_now, FMT)
    # Print total time yeast will be growing for
    if print_total_elapsed:
        total_time = round(time_diff.seconds / 3600, 2)
        print("Yeast will be growing for {} hours\n".format(total_time))
    if include_lag:
        if isinstance(include_lag, bool):
            time_diff += timedelta(hours=2) 
        elif isinstance(include_lag, Number):
            time_diff += timedelta(hours=include_lag)
        else:
            print("include_lag must either be a number of hours, or True (2 hour lag assumed); returning non-lagged calculation")
    number_of_doublings = (time_diff.seconds / 60) / doubling_time
    beginning_od = target_od / (2**(number_of_doublings))
    volume_of_starter = (beginning_od * target_volume) / starter_od
    volume_of_media = target_volume - volume_of_starter
    print('To make a {} mL culture at OD {} at {}:\n'.format(target_volume, target_od, target_time))
    print('Add {:0.3f} mL of starter at OD {} to {:0.3f} mL of media'.format(volume_of_starter,
                                                                             starter_od,
                                                                             volume_of_media))

In [91]:
how_should_i_innoculate(0.1, "17:00", 1000, 4, "8:00", print_total_elapsed=True)

Yeast will be growing for 9.0 hours
To make a 1000 mL culture at OD 0.1 at 17:00:

Add 0.391 mL of starter at OD 4 to 999.609 mL of media


In [92]:
how_should_i_innoculate(0.1, "10:00", 1000, 5, "22:40", print_total_elapsed=True)

Yeast will be growing for 11.33 hours
To make a 1000 mL culture at OD 0.1 at 10:00:

Add 0.106 mL of starter at OD 5 to 999.894 mL of media


In [84]:
test_time = datetime.strptime("17:00", "%H:%M") - datetime.strptime("10:00", "%H:%M")
test_time.seconds / 3600

timedelta(hours=2)

datetime.timedelta(0, 7200)

In [115]:
how_should_i_innoculate(0.1, "10:00", 1000, 5, "22:40", print_total_elapsed=True)

Yeast will be growing for 11.33 hours

To make a 1000 mL culture at OD 0.1 at 10:00:

Add 0.106 mL of starter at OD 5 to 999.894 mL of media
